# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [27]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [8]:
#identify categorical columns and numercial columns
categorical_columns = spaceship.select_dtypes(include=['object']).columns
numerical_columns = spaceship.select_dtypes(include=['float64', 'int64']).columns



In [11]:
categorical_columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP',
       'Name'],
      dtype='object')

In [12]:
#one hot encoding
spaceship = pd.get_dummies(spaceship, columns=categorical_columns, drop_first=True)
spaceship.head()


,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,PassengerId_0002_01,PassengerId_0003_01,PassengerId_0003_02,...,Name_Zinopus Spandisket,Name_Zinor Axlentindy,Name_Zinor Proorbeng,Name_Zinoth Lansuffle,Name_Zosmark Trattle,Name_Zosmark Unaasor,Name_Zosmas Ineedeve,Name_Zosmas Mormonized,Name_Zubeneb Flesping,Name_Zubeneb Pasharne
0,39.0,0.0,0.0,0.0,0.0,0.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,24.0,109.0,9.0,25.0,549.0,44.0,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,58.0,43.0,3576.0,0.0,6715.0,49.0,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,33.0,0.0,1283.0,371.0,3329.0,193.0,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,16.0,303.0,70.0,151.0,565.0,2.0,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


**Perform Train Test Split**

In [6]:
#splitting the data
X = spaceship.drop('Transported', axis=1)
y = spaceship['Transported']


In [16]:
#split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [21]:
#bagging and pasting
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Initialize the base model
base_model = DecisionTreeClassifier(random_state=0)

# Initialize the bagging classifier
bagging = BaggingClassifier(base_model, n_estimators=100, random_state=0)

# Fit the model
bagging.fit(X_train, y_train)

# Predict the test set
y_pred = bagging.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

ValueError: could not convert string to float: 'Earth'

- Random Forests

In [26]:
# Initialize the random forest classifier
rf = RandomForestRegressor(n_estimators=100, random_state=0)

# Fit the model
rf.fit(X_train, y_train)

# Predict the test set
y_pred = rf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

print(accuracy)


ValueError: could not convert string to float: 'Earth'

- Gradient Boosting

In [23]:
#gradient boosting
# Initialize the gradient boosting classifier
gb = GradientBoostingClassifier(n_estimators=100, random_state=0)

# Fit the model
gb.fit(X_train, y_train)

# Predict the test set
y_pred = gb.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

print(accuracy)


ValueError: could not convert string to float: 'Earth'

- Adaptive Boosting

In [24]:
#adaptive boosting
# Initialize the adaptive boosting classifier
ab = AdaBoostClassifier(n_estimators=100, random_state=0)

# Fit the model
ab.fit(X_train, y_train)

# Predict the test set
y_pred = ab.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

print(accuracy)


ValueError: Input contains NaN

Which model is the best and why?

In [ ]:
#comment here